In [20]:
import boto3, re
from sagemaker import get_execution_role
from tensorflow.python.saved_model import builder
from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
from tensorflow.python.saved_model import tag_constants
import keras
from keras.models import model_from_json
from keras import backend as K
import tarfile
import sagemaker

role = get_execution_role()

In [10]:
json_file = open('files/'+'model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

loaded_model.load_weights('files/model-weights.h5')

print("Loaded model from disk")

Loaded model from disk


In [11]:
# Note: This directory structure will need to be followed - see notes for the next section
model_version = '1'
export_dir = 'files/export/Servo/' + model_version

# Build the Protocol Buffer SavedModel at 'export_dir'
builder = builder.SavedModelBuilder(export_dir)

# Create prediction signature to be used by TensorFlow Serving Predict API
signature = predict_signature_def(
    inputs={"inputs": loaded_model.input}, outputs={"score": loaded_model.output})

with K.get_session() as sess:
    # Save the meta graph and variables
    builder.add_meta_graph_and_variables(
        sess=sess, tags=[tag_constants.SERVING], signature_def_map={"serving_default": signature})
    builder.save()

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: files/export/Servo/1/saved_model.pb


In [18]:
!ls files/export/Servo/1/variables

variables.data-00000-of-00001  variables.index


In [22]:
with tarfile.open('model.tar.gz', mode='w:gz') as archive:
    archive.add('files/export', recursive=True)

In [23]:
sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='model')

In [24]:
!touch train.py

In [28]:
from sagemaker.tensorflow.model import TensorFlowModel
sagemaker_model = TensorFlowModel(model_data = 's3://' + sagemaker_session.default_bucket() + '/model/model.tar.gz',
                                  role = role,
                                  framework_version = '1.12',
                                  entry_point = 'train.py')

The Python 2 tensorflow images will be soon deprecated and may not be supported for newer upcoming versions of the tensorflow images.
Please set the argument "py_version='py3'" to use the Python 3 tensorflow image.


In [ ]:
%%time
predictor = sagemaker_model.deploy(initial_instance_count=1,
                                   instance_type='ml.m4.xlarge')

-------------------------------------------------------------------------------------------------------------------------------------------------------------------*

UnexpectedStatusException: Error hosting endpoint sagemaker-tensorflow-2019-11-24-21-10-21-488: Failed. Reason:  The primary container for production variant AllTraffic did not pass the ping health check. Please check CloudWatch logs for this endpoint..

In [30]:
import numpy as np

# The sample model expects an input of shape [1,50]
data = np.random.randn(1, 50)
predictor.predict(data) 

NameError: name 'predictor' is not defined